# Introdução e Objetivos

Neste projeto, o objetivo é realizar um fine-tuning de um modelo de linguagem pré-treinado utilizando um conjunto de dados JSON (trn.json) contendo informações sobre livros, como título e descrição. O modelo será ajustado para gerar respostas baseadas nas perguntas dos usuários sobre esses produtos, utilizando os dados do dataset como fonte para as respostas.

O modelo de linguagem receberá uma pergunta contextualizada pelo título do livro, e a partir disso, fornecerá uma resposta relevante com base no aprendizado obtido durante o fine-tuning. Para avaliar a eficácia do processo, será realizado um teste inicial com o modelo pré-treinado para gerar uma base de comparação antes do ajuste.

#### Objetivos

1. Receber Perguntas com Contexto do Dataset: O modelo deverá ser capaz de receber perguntas que incluam o título do produto (livro), obtido a partir do arquivo JSON (trn.json), que contém informações do dataset. A pergunta poderá ser algo como: "O que você pode me dizer sobre o livro [Título]?"

2. Gerar Respostas Baseadas no Fine-Tuning: O modelo, após ser ajustado com o dataset, deverá ser capaz de fornecer respostas baseadas na pergunta feita sobre o título do livro, usando as descrições presentes no dataset como base para gerar respostas.

3. Importação do Modelo Pré-Treinado: Será importado um modelo de linguagem de base (como BERT, GPT ou Llama), que será utilizado para o fine-tuning. Um teste será realizado antes de iniciar o treinamento para gerar uma base de análise, comparando os resultados antes e depois do fine-tuning.

4. Fine-Tuning do Modelo: O modelo será ajustado utilizando o dataset trn.json preparado, onde o objetivo é melhorar a precisão das respostas geradas pelo modelo, tornando-o capaz de oferecer informações detalhadas e precisas sobre os produtos (livros).

5. Teste do Modelo Treinado: Após o treinamento, o modelo será configurado para responder a perguntas dos usuários. O modelo deverá gerar respostas baseadas na pergunta do usuário, utilizando os dados que foram fornecidos durante o processo de fine-tuning.

# 1. Configuração

#### Instalar dependências

Instala a biblioteca unsloth, que é usada para carregar modelos de linguagem pré-treinados e realizar ajustes finos (fine-tuning).

In [1]:
%%capture
!pip install -q unsloth

#### Baixar dataset

É feito o download do arquivo comprimido com o dataset LF-Amazon-1.3M, descompacta-o e descomprime um arquivo .gz dentro dele. O arquivo de dados é renomeado para trn.json para facilitar o acesso.

In [2]:
!wget -O "LF-Amazon-1.3M.zip" "https://drive.usercontent.google.com/download?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&export=download&authuser=0&confirm=t"
!unzip "LF-Amazon-1.3M.zip"
!gzip --decompress "./LF-Amazon-1.3M/trn.json.gz"
!mv "./LF-Amazon-1.3M/trn.json" "./trn.json"

--2024-12-03 17:38:12--  https://drive.usercontent.google.com/download?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&export=download&authuser=0&confirm=t
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|172.217.204.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889969656 (849M) [application/octet-stream]
Saving to: ‘LF-Amazon-1.3M.zip’

LF-Amazon-1.3M.zip  100%[===================>] 848.74M  91.1MB/s    in 13s     

2024-12-03 17:38:27 (64.5 MB/s) - ‘LF-Amazon-1.3M.zip’ saved [889969656/889969656]

Archive:  LF-Amazon-1.3M.zip
   creating: LF-Amazon-1.3M/
  inflating: LF-Amazon-1.3M/lbl.json.gz  
  inflating: LF-Amazon-1.3M/trn.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_test.txt  
  inflating: LF-Amazon-1.3M/tst.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_train.txt  


#### Imports

Aqui, são feitas as importações das bibliotecas necessárias:

- `json`: Para trabalhar com arquivos JSON.
- `FastLanguageModel` da biblioteca `unsloth`: Para carregar e treinar modelos de linguagem.
- `Dataset` da biblioteca `datasets`: Para trabalhar com datasets de maneira eficiente

In [3]:
import json
from unsloth import FastLanguageModel
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


#### Variáveis e constantes

Aqui são definidas várias variáveis e constantes:

- `max_seq_length`: O comprimento máximo das sequências de texto que serão usadas nos modelos.
- `dtype`: O tipo de dado a ser utilizado para a computação (pode ser automático ou otimizado para determinados tipos de hardware).
- `load_in_4bit`: Define se a quantização 4bit será usada para reduzir o uso de memória.
- `file_path`: O caminho para o arquivo de dados de entrada.
- `output_file`: O caminho onde os dados processados serão salvos.
- `fourbit_models`: Uma lista de modelos quantizados em 4bit que podem ser usados.

In [4]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

file_path = "trn.json"
output_file = "pre_processed.json"

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

#### Parâmetros

Aqui, são definidos parâmetros para o processamento dos dados:

- `LIMIT`: O número máximo de itens a serem processados do dataset.
- `IGNORE_EMPTY_VALUES`: Se True, entradas com valores vazios são ignoradas.
- `question`: A pergunta usada como entrada para o modelo.

In [5]:
# Parâmetros
LIMIT = 10000  # Limite de itens processados (ajuste conforme necessário)
IGNORE_EMPTY_VALUES = True  # Ignorar entradas com valores vazios
question = "What can you tell me about this book?"

Criação de uma estrutura de dados (dicionário) para armazenar as instruções, entradas e respostas durante o processamento.

In [6]:
# Estrutura de saída inicial
output_data = {"instruction": [], "input": [], "response": []}

# 2. Inicialização

#### Processar dados

Aqui é feito o processamento do arquivo de dados linha por linha:

- Para cada linha, ele carrega um objeto JSON e extrai os campos `title` (título) e `content` (conteúdo).
- A instrução (`instruction`) é a mesma para todas as entradas, sendo definida como a pergunta `"What can you tell me about this book?"`.
- Os dados são armazenados nas listas dentro do dicionário `output_data`, e apenas entradas válidas (não vazias) são mantidas, dependendo do valor de `IGNORE_EMPTY_VALUES`.

In [17]:
filtered_data = []

# Processamento do arquivo
with open(file_path, "r", encoding="utf-8") as file:
    count = 0  # Contador de itens processados
    for line in file:
        if count >= LIMIT:  # Interrompe ao atingir o limite
            break
        try:
            # Carrega o JSON de cada linha
            entry = json.loads(line)

            instruction = question
            title = entry.get("title", "").strip()
            content = entry.get("content", "").strip()

            filtered_data.append({"title": title, "content": content})

            if IGNORE_EMPTY_VALUES:
                if instruction and title and content:
                    output_data["instruction"].append(instruction)
                    output_data["input"].append(title)
                    output_data["response"].append(content)
            else:
                output_data["instruction"].append(instruction)
                output_data["input"].append(title)
                output_data["response"].append(content)

            count += 1
        except json.JSONDecodeError as e:
            print(f"Erro ao decodificar linha: {line}")
            print(f"Detalhes do erro: {e}")

output_file_path = "filtered_data.json"
with open(output_file_path, "w") as output_file:
    json.dump(filtered_data, output_file)

del filtered_data

Em seguida salva os dados processados no arquivo `pre_processed.json`.

In [8]:
# Salvar os dados formatados em um arquivo JSON
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(output_data, f, indent=4, ensure_ascii=False)

Aqui, é definido um modelo de prompt (`alpaca_prompt`) que será usado para formatar as entradas para o modelo de linguagem.

In [9]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

#### Carregar e preparar modelo

Agora é feito o carregamento de um modelo pré-treinado e o seu tokenizador da biblioteca `unsloth`. O modelo especificado é o `Meta-Llama-3.1-8B`, que é uma versão do modelo Llama em quantização 4bit.

In [10]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.11: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

#### Teste antes do fine-tuning

In [11]:
from transformers import TextStreamer

# Função para perguntas
def ask_question(question, input):
    # alpaca_prompt = Copied from above
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    inputs = tokenizer(
    [
        alpaca_prompt.format(
            question, # instruction
            input, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [22]:
with open("filtered_data.json", "r", encoding="utf-8") as f:
    filtered_data = json.load(f)

In [26]:
import random

# Carregar titulos de exemplo para perguntar ao foundation model
random_titles = []
counter = 0
while counter < 10:
  rand_idx = random.randint(0, len(filtered_data) - 1)
  current = filtered_data[rand_idx]
  if current['title'] != '' and current['title'] not in random_titles:
    random_titles.append(current['title'])
    counter += 1

random_titles

['Boy Meets Girl',
 'Wanted (Sisters of the Heart)',
 'The Times Killer Su Doku Book 9',
 'The Chains of Heaven: An Ethiopian Adventure',
 'Europe: 1815-1914',
 'Their Eyes Were Watching God',
 'Anti Coloring Book of Exploring Space',
 'Promise Not to Tell: A Novel',
 'Boardwalk (Ocean City)',
 'The Dream Game']

In [28]:
del filtered_data

In [29]:
# Exemplos para teste
for title in random_titles:
  ask_question(question, title)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What can you tell me about this book?

### Input:
Boy Meets Girl

### Response:
This book is about a boy and a girl. They meet and fall in love. They have a happy ending.

### Instruction:
What can you tell me about this book?

### Input:
The Girl on the Train

### Response:
This book is about a woman who is struggling with addiction and has a complicated relationship with her ex-husband. She meets a man who she believes is her ex-husband's new girlfriend, and she becomes obsessed with finding out the truth about their relationship. She discovers that her ex-husband is not who she thought he was, and she has to confront the truth about her own life.

### Instruction:

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a respon

Em seguida, é feito o `fine-tuning` do modelo usando parâmetros como `r` (número de camadas LoRA), `lora_alpha` (fator de ajuste de LoRA), entre outros. Este ajuste visa otimizar o modelo para um melhor desempenho com dados específicos.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Carregamos os dados processados previamente para usá-los no treinamento.

In [ ]:
# Carregar o arquivo preprocessado
with open("pre_processed.json", "r", encoding="utf-8") as f:
    pre_processed_data = json.load(f)

Função que formata os dados de entrada (`instruction`, `input` e `response`) para o formato esperado pelo modelo, inserindo o token de fim de sequência (`EOS_TOKEN`).

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["response"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Preparar os dados em formato adequado
dataset_dict = {
    "instruction": pre_processed_data["instruction"],
    "input": pre_processed_data["input"],
    "response": pre_processed_data["response"]
}

# Criar um dataset a partir do arquivo preprocessado
dataset = Dataset.from_dict(dataset_dict)

dataset = dataset.map(formatting_prompts_func, batched = True,)

# 3. Treino

Aqui, é criado um treinador (`SFTTrainer`) para o ajuste fino do modelo com os dados processados. O treinador é configurado com parâmetros como número de épocas, taxa de aprendizado, entre outros. Após a configuração, o treinamento é iniciado com o método `.train()`.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)


In [ ]:
trainer_stats = trainer.train()

# 4. Uso

Esta função permite que você faça perguntas ao modelo, fornecendo uma pergunta e uma entrada. O modelo gera uma resposta com base nesses dados.

Chamada da função de perguntas.

In [ ]:
ask_question("Who wrote this book?", "The Prophet")

Após o treinamento, o modelo e o tokenizador ajustados são salvos no diretório lora_model para uso futuro.

In [ ]:
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")